# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Business Problem
You are investigating the data collected from a homepage for a music app. We want to assess which website is better. 

Metric: 
- Interaction with the website (view/click)

In [1]:
# import required libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#load data
df = pd.read_csv('homepage_actions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [3]:
df.describe(include='all') # Explore continuous and catagorical)

,timestamp,id,group,action
count,8188,8188.000000,8188,8188
unique,8188,NaN,2,2
top,2017-01-10 23:39:33.837271,NaN,control,view
freq,1,NaN,4264,6328
mean,NaN,564699.749878,NaN,NaN
std,NaN,219085.845672,NaN,NaN
min,NaN,182988.000000,NaN,NaN
25%,NaN,373637.500000,NaN,NaN
50%,NaN,566840.500000,NaN,NaN
75%,NaN,758078.000000,NaN,NaN


In [4]:
df.tail()

,timestamp,id,group,action
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view
8187,2017-01-18 10:24:08.629327,461199,control,view


In [5]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [6]:
len(df[df['action'] == 'click'])

1860

In [7]:
# Complete duplicated rows
df.duplicated().sum()

0

In [8]:
# Duplicated id
df.duplicated(subset=['id']).sum()
#ids = df["id"]
#len(df[ids.isin(ids[ids.duplicated()])].sort_values("id"))

1860

In [9]:
# Any missing data?
df.isna().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [10]:
# Are there any anomalies with the data; did anyone click who didn't view?
viewed_id = df.loc[df['action'] == 'view', 'id']
clicked_id = df.loc[df['action'] == 'click', 'id']

anomaly = len(clicked_id[clicked_id.isin(~viewed_id)]) # are there any clickers ids not in viewers?

print("Number of Clickers who didn't view: {}".format(anomaly))


Number of Clickers who didn't view: 0


In [11]:
# How many viewers also clicked?
anomaly = len(viewed_id[viewed_id.isin(clicked_id)]) # are there any clickers ids not in viewers?

print("Number of Viewers who also clicked: {}".format(anomaly))

Number of Viewers who also clicked: 1860


In [12]:
# Is there any overlap between the control and experiment groups?

# Subset the data based on Control and Experiment website
website_con_id = df.loc[df['group'] == 'control', 'id']
website_exp_id = df.loc[df['group'] == 'experiment', 'id']

overlap = len(website_exp_id[website_exp_id.isin(website_con_id)]) # any same ids in control and exp groups?
print("Number of overlap between experiment and control groups: {}".format(overlap))

Number of overlap between experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

### What statistical test to use?

#### Data have
- Count the number of users that viewed and clicked (1) versus only viewed (0).
- Compare the counts between the two groups (websites)
- Use a poisson

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

In [13]:
#Subset the data into control and experimental groups
website_con = df.loc[df['group'] == 'control']
website_exp = df.loc[df['group'] == 'experiment']

# Calculate the binary value of only viewed (0) or clicked (1)
control_clicked = website_con.duplicated(subset=['id']).sum()
control_total = len(set(website_con['id'].unique()))
control_view_only = control_total - control_clicked

exp_clicked = website_exp.duplicated(subset=['id']).sum()
exp_total = len(set(website_exp['id'].unique()))
exp_view_only = exp_total - exp_clicked

print("Total Observations in control and experimental groups: {}".format(control_total + exp_total))
print("Total number of visitors in control group: {}".format(control_total))
print("Total number of clickers in control group: {}\n".format(control_clicked))
print("Total number of visitors in experimental group: {}".format(exp_total))
print("Total number of clickers in experimental group: {}".format(exp_clicked))

Total Observations in control and experimental groups: 6328
Total number of visitors in control group: 3332
Total number of clickers in control group: 932

Total number of visitors in experimental group: 2996
Total number of clickers in experimental group: 928


### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [14]:
expected_clicks = exp_total * (control_clicked / control_total)
print("Expected number of clicks in the experimental group: %.0f" %expected_clicks)
print("Observed number of clicks in the experimental group: %.0f" %exp_clicked)

Expected number of clicks in the experimental group: 838
Observed number of clicks in the experimental group: 928


### Step 2:
Calculate the Z-score (number of standard deviations) that the actual number of clicks was from this estimate.

In [15]:
# calculate variance
p = control_clicked / control_total
var = exp_total * p * (1-p)

# calculate standard deviation
stdev = np.sqrt(var)
print("Standard deviation: %.2f" %stdev)

# calcualte z-score
z_score = (exp_clicked - expected_clicks)/stdev
print("Z-score: %.2f" %z_score)

Standard deviation: 24.57
Z-score: 3.66


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [16]:
# Calculate p-value
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The p-value is less than the critical value of 0.05, thus we can reject the null hypothesis that the number of clicks in the experimental group is equal to the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.